In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold, train_test_split

ques 1

In [8]:
from google.colab import files
uploaded = files.upload()
house_data = pd.read_csv('USA_Housing.csv')
X = house_data.drop(columns=["Price"]).values
y = house_data["Price"].values

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

kf = KFold(n_splits=5, shuffle=True, random_state=42)

best_beta = None
best_r2 = -np.inf

fold = 1
for train_idx, test_idx in kf.split(X_scaled):
    X_train, X_test = X_scaled[train_idx], X_scaled[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    beta = np.linalg.inv(X_train.T @ X_train) @ (X_train.T @ y_train)

    y_pred = X_test @ beta
    r2 = r2_score(y_test, y_pred)

    print(f"Fold {fold} R2 Score: {r2:.4f}")
    fold += 1
    if r2 > best_r2:
        best_r2 = r2
        best_beta = beta

print("\nBest R2 Score from CV:", best_r2)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)
beta_final = np.linalg.inv(X_train.T @ X_train) @ (X_train.T @ y_train)

y_pred_final = X_test @ beta_final
print("Final R2 Score (70/30 split):", r2_score(y_test, y_pred_final))



Saving USA_Housing.csv to USA_Housing (2).csv
Fold 1 R2 Score: -11.4420
Fold 2 R2 Score: -11.8285
Fold 3 R2 Score: -11.6144
Fold 4 R2 Score: -10.9997
Fold 5 R2 Score: -10.5798

Best R2 Score from CV: -10.579758582041713
Final R2 Score (70/30 split): -12.003903385562284


ques 2

In [9]:
X_train_val, X_test, y_train_val, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.2, random_state=42)

def gradient_descent(X, y, lr, iterations=1000):
    m, n = X.shape
    beta = np.zeros(n)
    for _ in range(iterations):
        gradient = -(2/m) * (X.T @ (y - X @ beta))
        beta -= lr * gradient
    return beta

learning_rates = [0.001, 0.01, 0.1, 1]
best_beta = None
best_val_r2 = -np.inf

for lr in learning_rates:
    beta = gradient_descent(X_train, y_train, lr=lr, iterations=1000)
    y_val_pred = X_val @ beta
    y_test_pred = X_test @ beta

    r2_val = r2_score(y_val, y_val_pred)
    r2_test = r2_score(y_test, y_test_pred)

    print(f"\nLearning Rate {lr}:")
    print("Validation R2:", r2_val)
    print("Test R2:", r2_test)

    if r2_val > best_val_r2:
        best_val_r2 = r2_val
        best_beta = beta

print("\nBest Beta (from validation) coefficients:\n", best_beta)


Learning Rate 0.001:
Validation R2: -11.309840188168787
Test R2: -12.05506738296265

Learning Rate 0.01:
Validation R2: -11.318069140536533
Test R2: -12.010979529759709

Learning Rate 0.1:
Validation R2: -11.31806970348623
Test R2: -12.010979232167713

Learning Rate 1:
Validation R2: -inf
Test R2: -inf

Best Beta (from validation) coefficients:
 [196011.02066971 138171.30668778 117004.69583016  39664.25921488
 116553.34299286]


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_regression.py:1275: RuntimeWarning: overflow encountered in square
  numerator = xp.sum(weight * (y_true - y_pred) ** 2, axis=0)
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_regression.py:1275: RuntimeWarning: overflow encountered in square
  numerator = xp.sum(weight * (y_true - y_pred) ** 2, axis=0)


ques 3

In [10]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data"
columns = ["symboling", "normalized_losses", "make", "fuel_type", "aspiration","num_doors", "body_style",
           "drive_wheels", "engine_location", "wheel_base", "length", "width", "height", "curb_weight",
           "engine_type", "num_cylinders", "engine_size", "fuel_system", "bore", "stroke",
           "compression_ratio", "horsepower", "peak_rpm", "city_mpg", "highway_mpg", "price"]

df = pd.read_csv(url, names=columns, na_values='?')



In [11]:

df.head()
df.isna().sum()

,0
symboling,0
normalized_losses,41
make,0
fuel_type,0
aspiration,0
num_doors,2
body_style,0
drive_wheels,0
engine_location,0
wheel_base,0


In [12]:
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
numeric_cols.remove('price')

# for columns with numerical values and imputing them with mean
for col in numeric_cols:
    df[col].fillna(df[col].astype(float).mean(), inplace=True)

# # for columns with categorical values and imputing them with most occurent value (mode)
for col in categorical_cols:
    df[col].fillna(df[col].mode()[0], inplace=True)

/tmp/ipython-input-1416698875.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].astype(float).mean(), inplace=True)
/tmp/ipython-input-1416698875.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True

In [13]:
words_to_num = {
    'two': 2,
    'three': 3,
    'four': 4,
    'five': 5,
    'six': 6,
    'eight': 8,
    'twelve': 12
}

df['num_doors'] = df['num_doors'].map(words_to_num)
df['num_cylinders'] = df['num_cylinders'].map(words_to_num)

In [14]:
df = pd.get_dummies(df, columns=['body_style', 'drive_wheels'], drop_first=True)

In [15]:

from sklearn.preprocessing import LabelEncoder

label_cols = ['make', 'aspiration', 'engine_location', 'fuel_type']
le = LabelEncoder()

for col in label_cols:
    df[col] = le.fit_transform(df[col])

In [16]:
print(df.dtypes[df.dtypes == 'object'])

engine_type    object
fuel_system    object
dtype: object


In [17]:
df['fuel_system'] = df['fuel_system'].apply(lambda x: 1 if 'pfi' in x else 0)
df['engine_type'] = df['engine_type'].apply(lambda x: 1 if 'ohc' in x else 0)

In [18]:

from sklearn.preprocessing import StandardScaler

X = df.drop('price', axis=1)
y = df['price'].astype(float)  # convert price to float

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [19]:
print(df.isna().sum())  # Should print 0

symboling               0
normalized_losses       0
make                    0
fuel_type               0
aspiration              0
num_doors               0
engine_location         0
wheel_base              0
length                  0
width                   0
height                  0
curb_weight             0
engine_type             0
num_cylinders           0
engine_size             0
fuel_system             0
bore                    0
stroke                  0
compression_ratio       0
horsepower              0
peak_rpm                0
city_mpg                0
highway_mpg             0
price                   4
body_style_hardtop      0
body_style_hatchback    0
body_style_sedan        0
body_style_wagon        0
drive_wheels_fwd        0
drive_wheels_rwd        0
dtype: int64


In [22]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

y_pred = lr_model.predict(X_test)

print(f"R2 score on test set: {r2_score(y_test, y_pred):.4f}")
print(f"RMSE on test set: {np.sqrt(mean_squared_error(y_test, y_pred)):.4f}")

ValueError: Input y contains NaN.

In [21]:
from sklearn.decomposition import PCA

# Retain enough components to explain 95% variance (or choose number manually)
pca = PCA(0.95)  # 95% variance explained
X_pca = pca.fit_transform(X_scaled)

print(f"Original number of features: {X_scaled.shape[1]}")
print(f"Reduced number of features: {X_pca.shape[1]}")

X_train_pca, X_test_pca, y_train_pca, y_test_pca = train_test_split(X_pca, y, test_size=0.3, random_state=42)

lr_model_pca = LinearRegression()
lr_model_pca.fit(X_train_pca, y_train_pca)

y_pred_pca = lr_model_pca.predict(X_test_pca)

print(f"R2 score on PCA test set: {r2_score(y_test_pca, y_pred_pca):.4f}")
print(f"RMSE on PCA test set: {np.sqrt(mean_squared_error(y_test_pca, y_pred_pca)):.4f}")

Original number of features: 29
Reduced number of features: 16


ValueError: Input y contains NaN.